In [1]:
%reload_ext autoreload
%autoreload 2

In [18]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud


In [3]:
os.chdir('..')

In [4]:
from src.loader import NewsDataLoader

In [5]:
from src.config import cfg

Output File: news_output.csv
Path: data
News Data: rating.csv
Traffic Data: trafiic.csv
Domain Location Data: domains_location.csv


In [6]:
import src.utils as utils

In [26]:
news_data_loader = NewsDataLoader(cfg.path)

In [8]:
original_news_df = news_data_loader.get_news_data()

In [10]:
# check columns
original_news_df.columns

Index(['article_id', 'source_id', 'source_name', 'author', 'title',
       'description', 'url', 'url_to_image', 'published_at', 'content',
       'category', 'article', 'title_sentiment'],
      dtype='object')

In [12]:
original_news_df.isna().sum()

article_id             0
source_id          40585
source_name            0
author              2163
title                  0
description           10
url                    0
url_to_image        3451
published_at           0
content                0
category              21
article                0
title_sentiment        0
dtype: int64

In [41]:
test_content = original_news_df.head(10)['content'][9]
test_title = original_news_df.head(10)['title'][9]
print(test_title)

marine le pen found guilty of defamation after accusing french ngo of smuggling migrants in mayotte


In [43]:

# this is grat but we need vectorizers to get the keywords that sound grammatically correct
kw_extractor = yake.KeywordExtractor(n=1, dedupLim=0.4, top=10)
content_keywords = kw_extractor.extract_keywords(test_content)
title_keywords = kw_extractor.extract_keywords(test_title)

for kw in content_keywords:
    print(kw)


print('Title Keywords: \n')
for kw in title_keywords:
    print(kw)

('Cimade', 0.03874456518394107)
('Pen', 0.10141226884569679)
('French', 0.10141226884569679)
('NGO', 0.10141226884569679)
('Marine', 0.13696809108329627)
('migrants', 0.13696809108329627)
('smugglers', 0.13696809108329627)
('convicted', 0.2200560909321288)
('defamation', 0.2200560909321288)
('helping', 0.2200560909321288)
Title Keywords: 

('marine', 0.09568045026443411)
('pen', 0.15831692877998726)
('found', 0.15831692877998726)
('guilty', 0.15831692877998726)
('defamation', 0.15831692877998726)
('accusing', 0.15831692877998726)
('french', 0.15831692877998726)
('ngo', 0.15831692877998726)
('smuggling', 0.15831692877998726)
('migrants', 0.15831692877998726)
